In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import warnings
warnings.simplefilter("ignore") 
import logging
logging.basicConfig(level=logging.ERROR)

In [4]:
from kinoml.datasets.chembl import ChEMBLDatasetProvider
chembl = ChEMBLDatasetProvider.from_source(measurement_types=("IC50",))

RDKit WARNING: [18:36:32] Enabling RDKit 2019.09.2 jupyter extensions
100%|██████████| 170458/170458 [08:18<00:00, 341.90it/s] 


In [5]:
chembl

<ChEMBLDatasetProvider with 170458 IC50Measurement measurements and 141680 systems>

In [6]:
df = chembl.to_dataframe()
df

,Systems,n_components,Measurement,MeasurementType
0,P00533 & Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F...,2,41.0,IC50Measurement
1,P35968 & Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F...,2,16500.0,IC50Measurement
2,P00533 & Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)...,2,170.0,IC50Measurement
3,P06239 & Nc1ncnc2c1c(-c1cccc(Oc3ccccc3)c1)cn2C...,2,140.0,IC50Measurement
4,P06239 & Nc1ncnc2c1c(-c1cccc(Oc3ccccc3)c1)cn2C...,2,1180.0,IC50Measurement
...,...,...,...,...
170453,P36888 & CCC[C@H](C)Nc1ncc2c(-c3cnn(C4CCNCC4)c...,2,45.0,IC50Measurement
170454,Q12866 & CCC[C@H](C)Nc1ncc2c(-c3ccc4nc(C)oc4c3...,2,22.0,IC50Measurement
170455,P30530 & CCC[C@H](C)Nc1ncc2c(-c3ccc4nc(C)oc4c3...,2,3295.0,IC50Measurement
170456,Q06418 & CCC[C@H](C)Nc1ncc2c(-c3ccc4nc(C)oc4c3...,2,268.0,IC50Measurement


In [13]:
print("Measurements:", len(chembl.measurements))
print("Systems:", len(chembl.systems))
print("Proteins:", len(set([s.protein for s in chembl.systems])))
print("Ligands:",len(set([s.ligand.name for s in chembl.systems])))

Measurements: 170458
Systems: 141680
Proteins: 347
Ligands: 93855


Having this many ligands (compared to PKIS2) makes this dataset take much more memory and longer (~10 mins) to initialize!